In [4]:
import torch as torch
import warnings
warnings.simplefilter('ignore')

from time import time
from datasets.BrainImagingDataset_3D import BrainImgDataset
from  monai import transforms
import numpy as np
from collections.abc import Callable, Sequence
from torch.utils.data import Dataset as _TorchDataset
from torch.utils.data import Subset
import collections
import numpy as np
from monai.data import *
from monai.transforms import *
import argparse
import pickle
from utils.brain_data_utils import threshold

import torch.optim as optim
from models.voco_head import VoCoHead
from optimizers.lr_scheduler import WarmupCosineSchedule
from torch.cuda.amp import GradScaler, autocast

from utils.data_utils import get_position_label, get_crop_transform, get_vanilla_transform, VoCoAugmentation
from utils.utils import AverageMeter

import torch as torch
# from lightning import LightningModule, LightningDataModule, Trainer, seed_everything
# from lightning.loggers import WandbLogger
import warnings
import os
import sys
# from lightning.callbacks import ModelCheckpoint, TQDMProgressBar, EarlyStopping,  LearningRateMonitor

# from lightning.loggers import WandbLogger
import wandb
import monai.transforms as transforms
from monai import data
import argparse


import torch.optim as optim
from models.voco_head import VoCoHead
from optimizers.lr_scheduler import WarmupCosineSchedule
from torch.utils.tensorboard import SummaryWriter
from utils.data_utils import *
from utils.ops import *

parser = argparse.ArgumentParser(description="Swin UNETR segmentation pipeline")
args, unknown = parser.parse_known_args()


args.roi = 142
args.num_workers = 1
args.logdir = "logs"
args.epochs = 10
args.num_steps = 100  #
args.eval_num = 100
args.warmup_steps = 5000
args.in_channels = 1
args.feature_size = 48 #

args.dropout_path_rate = 0.0
args.use_checkpoint = True
args.spatial_dims = 3
args.a_min = -175.0
args.a_max = 250.0
args.b_min = 0.0
args.b_max = 1.0
args.space_x = 1.5
args.space_y = 1.5
args.space_z = 1.5
args.roi_x = roi
args.roi_y = roi
args.roi_z = roi
args.batch_size = 1
args.sw_batch_size = 2
args.lr = 1e-4
args.decay = 0.1
args.momentum = 0.9
args.lrdecay = True
args.max_grad_norm = 1.0
args.loss_type = "SSL"
args.opt = "adamw"
args.lr_schedule = "warmup_cosine"





paths  = {    'root_dir':  '/data2/biodata/biobank/brain_data/',
    'table_dir' : '/data2/biodata/biobank/tables/'}
root_dir =  paths['root_dir']
modality_path = 'T1/T1_brain_to_MNI.nii.gz'
table_dir = paths['table_dir']

split = 0.002
batch_size=1
num_workers=4

datalist = BrainImgDataset(root_dir, modality_path,table_dir, percent_split = split,split='train').dict_paths
val_files = BrainImgDataset(root_dir, modality_path,table_dir,percent_split = split, split='val').dict_paths




train_transforms = transforms.Compose([LoadImaged(keys=["image"], image_only=True, dtype=np.int16),
                                EnsureChannelFirstd(keys=["image"]),
                                Orientationd(keys=["image"], axcodes="RAS"),
                                ScaleIntensityRanged(
                                    keys=["image"], a_min=args.a_min, a_max=args.a_max,
                                    b_min=args.b_min, b_max=args.b_max, clip=True),
                                SpatialPadd(keys="image", spatial_size=[args.roi_x, args.roi_y,
                                                                        args.roi_z]),
                                CropForegroundd(keys=["image"], source_key="image"),
                                SpatialCropd(keys=["image"], roi_start=[60, 80, 0],
                                             roi_end=[440, 380, 10000]),
                                Resized(keys=["image"], mode="trilinear", align_corners=True,
                                        spatial_size=(384, 384, 96)),

                                # Random
                                RandShiftIntensityd(keys="image", offsets=0.1, prob=0.0),
                                CropForegroundd(keys="image", source_key="image", select_fn=threshold),
                                Resized(keys="image", mode="bilinear", align_corners=True,
                                        spatial_size=(384, 384, 96)),

                                VoCoAugmentation(args, aug=True)
                                ])

val_transforms = transforms.Compose([LoadImaged(keys=["image"], image_only=True, dtype=np.int16),
                              EnsureChannelFirstd(keys=["image"]),
                              Orientationd(keys=["image"], axcodes="RAS"),
                              ScaleIntensityRanged(
                                  keys=["image"], a_min=args.a_min, a_max=args.a_max,
                                  b_min=args.b_min, b_max=args.b_max, clip=True),
                              SpatialPadd(keys="image", spatial_size=[args.roi_x, args.roi_y,
                                                                      args.roi_z]),
                              CropForegroundd(keys=["image"], source_key="image"),
                              SpatialCropd(keys=["image"], roi_start=[60, 80, 0],
                                           roi_end=[440, 380, 10000]),
                              Resized(keys=["image"], mode="trilinear", align_corners=True,
                                      spatial_size=(384, 384, 96)),
                              VoCoAugmentation(args, aug=False)
                              ])





train_ds = data.PersistentDataset(data=datalist,
                                     transform=train_transforms,
                                     pickle_protocol=pickle.HIGHEST_PROTOCOL,
                                     cache_dir='/data/saved/voco_cache')
train_sampler = None
train_loader = data.DataLoader(
        train_ds, batch_size=batch_size, num_workers=num_workers, sampler=None, shuffle=True,
        drop_last=True, pin_memory=True
    )



model = VoCoHead(args)

model.train()
loss_train = []
run_loss = AverageMeter()
pos_avg, neg_avg, base_avg = AverageMeter(), AverageMeter(), AverageMeter()
torch.cuda.set_device(0)
model.cuda()

def concat_image(imgs):
    output = []
    for img in imgs:
        img = img['image']
        output.append(img)
        
    output = torch.concatenate(output, dim=1)

    bs, sw_s, x, y, z = output.size()
    output = output.view(-1, 1, x, y, z)
    return output

for step, batch in enumerate(train_loader):
    t1 = time()
    img, labels, crops = batch
    print(step, len(batch))

    img, crops = concat_image(img), concat_image(crops)




    # print(img.size(), crops.size(), labels.size())
    img, crops, labels = img.cuda(), crops.cuda(), labels.cuda()

    with autocast(enabled=False):
        pos, neg, b_loss = model(img, crops, labels)
        loss = pos + neg + b_loss
        loss_train.append(loss.item())

whole dataset size 23209
Number of train images
34
Number of  train images  34
whole dataset size 23209
Number of val images
7
Number of  val images  7
0 3


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.15 GiB. GPU 0 has a total capacity of 11.62 GiB of which 1.08 GiB is free. Process 3154473 has 7.06 GiB memory in use. Including non-PyTorch memory, this process has 3.42 GiB memory in use. Of the allocated memory 3.18 GiB is allocated by PyTorch, and 18.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [2]:
import torch as torch
import warnings
warnings.simplefilter('ignore')

from datasets.BrainImagingDataset_3D import BrainImgDataset
from  monai import transforms
import numpy as np
from collections.abc import Callable, Sequence
from torch.utils.data import Dataset as _TorchDataset
from torch.utils.data import Subset
import collections
import numpy as np
from monai import data
import argparse
import pickle
from utils.brain_data_utils import threshold

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
paths  = {    'root_dir':  '/data2/biodata/biobank/brain_data/',
    'table_dir' : '/data2/biodata/biobank/tables/'}
root_dir =  paths['root_dir']
modality_path = 'T1/T1_brain_to_MNI.nii.gz'
table_dir = paths['table_dir']

split = 0.2

datalist = BrainImgDataset(root_dir, modality_path,table_dir, percent_split = split,split='train').dict_paths
val_files = BrainImgDataset(root_dir, modality_path,table_dir,percent_split = split, split='val').dict_paths

whole dataset size 23209
Number of train images
3480
Number of  train images  3480
whole dataset size 23209
Number of val images
696
Number of  val images  696


In [5]:
roi = 64
num_workers = 1
logdir = "logs"
epochs = 100
num_steps = 250000
eval_num = 100
warmup_steps = 5000
in_channels = 1
feature_size = 48
dropout_path_rate = 0.0
use_checkpoint = True
spatial_dims = 3
a_min = -175.0
a_max = 250.0
b_min = 0.0
b_max = 1.0
space_x = 1.5
space_y = 1.5
space_z = 1.5
roi_x = roi
roi_y = roi
roi_z = roi
batch_size = 1
sw_batch_size = 2
lr = 1e-4
decay = 0.1
momentum = 0.9
lrdecay = True
max_grad_norm = 1.0
loss_type = "SSL"
opt = "adamw"
lr_schedule = "warmup_cosine"

In [6]:
class VoCoAugmentation():
    def __init__(self,  aug):

        self.aug = aug

    def __call__(self, x_in):
        crops_trans = get_crop_transform(roi_small=roi_x, aug=self.aug)

        vanilla_trans, labels = get_vanilla_transform(num=sw_batch_size,
                                                      roi_small=roi_x, aug=self.aug)

        imgs = []
        for trans in vanilla_trans:
            img = trans(x_in)
            imgs.append(img)

        crops = []
        for trans in crops_trans:
            crop = trans(x_in)
            crops.append(crop)

        return imgs, labels, crops


def get_vanilla_transform(num=2, num_crops=4, roi_small=64, roi=96, max_roi=384, aug=False):
    vanilla_trans = []
    labels = []
    for i in range(num):
        center_x, center_y, label = get_position_label(roi=roi,
                                                       max_roi=max_roi,
                                                       num_crops=num_crops)
        if aug:
            trans = transforms.Compose([
               transforms.SpatialCropd(keys=['image'],
                             roi_center=[center_x, center_y, roi // 2],
                             roi_size=[roi, roi, roi]),
               transforms.Resized(keys=["image"], mode="bilinear", align_corners=True,
                        spatial_size=(roi_small, roi_small, roi_small)),
               transforms.RandFlipd(keys=["image"], prob=0.2, spatial_axis=0),
               transforms.RandFlipd(keys=["image"], prob=0.2, spatial_axis=1),
               transforms.RandFlipd(keys=["image"], prob=0.2, spatial_axis=2),
                transforms.RandRotate90d(keys=["image"], prob=0.2, max_k=3),
                transforms.RandShiftIntensityd(keys="image", offsets=0.1, prob=0.1),
                transforms.ToTensord(keys=["image"])])
        else:
            trans = transforms.Compose([
                transforms.SpatialCropd(keys=['image'],
                             roi_center=[center_x, center_y, roi // 2],
                             roi_size=[roi, roi, roi]),
               transforms.Resized(keys=["image"], mode="bilinear", align_corners=True,
                        spatial_size=(roi_small, roi_small, roi_small)),
                transforms.ToTensord(keys=["image"])])

        vanilla_trans.append(trans)
        labels.append(label)

    labels = np.concatenate(labels, 0).reshape(num, num_crops * num_crops)


    return vanilla_trans, labels


def get_crop_transform(num=4, roi_small=64, roi=96, aug=False):
    voco_trans = []
    # not symmetric at axis x !!!
    for i in range(num):
        for j in range(num):
            center_x = (i + 1 / 2) * roi
            center_y = (j + 1 / 2) * roi
            center_z = roi // 2

            if aug:
                trans = transforms.Compose([
                    transforms.SpatialCropd(keys=['image'],
                                 roi_center=[center_x, center_y, center_z],
                                 roi_size=[roi, roi, roi]),
                   transforms.Resized(keys=["image"],
                            mode="bilinear",
                            align_corners=True,
                            spatial_size=(roi_small, roi_small, roi_small)
                            ),
                   transforms.Resized(keys=["image"], mode="bilinear", align_corners=True,
                            spatial_size=(roi_small, roi_small, roi_small)),
                    transforms.RandFlipd(keys=["image"], prob=0.2, spatial_axis=0),
                    transforms.RandFlipd(keys=["image"], prob=0.2, spatial_axis=1),
                    transforms.RandFlipd(keys=["image"], prob=0.2, spatial_axis=2),
                    transforms.RandRotate90d(keys=["image"], prob=0.2, max_k=3),
                    transforms.RandShiftIntensityd(keys="image", offsets=0.1, prob=0.1),
                    transforms.ToTensord(keys=["image"])],
                )
            else:
                trans = transforms.Compose([
                    transforms.SpatialCropd(keys=['image'],
                                 roi_center=[center_x, center_y, center_z],
                                 roi_size=[roi, roi, roi]),
                   transforms.Resized(keys=["image"],
                            mode="bilinear",
                            align_corners=True,
                            spatial_size=(roi_small, roi_small, roi_small)
                            ),
                    transforms.ToTensord(keys=["image"])],
                )

            voco_trans.append(trans)

    return voco_trans


def get_position_label(roi=96, base_roi=96, max_roi=384, num_crops=4):
    half = roi // 2
    center_x, center_y = np.random.randint(low=half, high=max_roi - half), \
        np.random.randint(low=half, high=max_roi - half)
    # center_x, center_y = np.random.randint(low=half, high=half+1), \
    #     np.random.randint(low=half, high=half+1)
    # center_x, center_y = roi + half, roi + half
    # print(center_x, center_y)

    x_min, x_max = center_x - half, center_x + half
    y_min, y_max = center_y - half, center_y + half

    total_area = roi * roi
    labels = []
    for i in range(num_crops):
        for j in range(num_crops):
            crop_x_min, crop_x_max = i * base_roi, (i + 1) * base_roi
            crop_y_min, crop_y_max = j * base_roi, (j + 1) * base_roi

            dx = min(crop_x_max, x_max) - max(crop_x_min, x_min)
            dy = min(crop_y_max, y_max) - max(crop_y_min, y_min)
            if dx <= 0 or dy <= 0:
                area = 0
            else:
                area = (dx * dy) / total_area
            labels.append(area)

    labels = np.asarray(labels).reshape(1, num_crops * num_crops)

    return center_x, center_y, labels


if __name__ == '__main__':
    center_x, center_y, labels = get_position_label()
    print(center_x, center_y, labels)

316 69 [[0.         0.         0.         0.         0.         0.
  0.         0.         0.16276042 0.04557292 0.         0.
  0.61848958 0.17317708 0.         0.        ]]


In [7]:




transforms.Compose([transforms.LoadImaged(keys=["image"], image_only=True, dtype=np.int16),
                                transforms.EnsureChannelFirstd(keys=["image"]),
                                transforms.Orientationd(keys=["image"], axcodes="RAS"),
                                transforms.ScaleIntensityRanged(
                                    keys=["image"], a_min=a_min, a_max=a_max,
                                    b_min=b_min, b_max=b_max, clip=True),
                                transforms.SpatialPadd(keys="image", spatial_size=[roi_x, roi_y,
                                                                        roi_z]),
                                transforms.CropForegroundd(keys=["image"], source_key="image"),
                                transforms.SpatialCropd(keys=["image"], roi_start=[60, 80, 0],
                                             roi_end=[440, 380, 10000]),
                                transforms.Resized(keys=["image"], mode="trilinear", align_corners=True,
                                        spatial_size=(384, 384, 96)),

                                # Random
                                transforms.RandShiftIntensityd(keys="image", offsets=0.1, prob=0.0),
                                transforms.CropForegroundd(keys="image", source_key="image", select_fn=threshold),
                                transforms.Resized(keys="image", mode="bilinear", align_corners=True,
                                        spatial_size=(384, 384, 96)),

                                VoCoAugmentation( aug=True)
                                ])

train_transforms = transforms.Compose([ transforms.LoadImaged(keys=["image"], image_only=True, dtype=np.int16),
                             transforms.EnsureChannelFirstd(keys=["image"]),
                             transforms.Orientationd(keys=["image"], axcodes="RAS"),
                             transforms.ScaleIntensityRanged(
                                keys=["image"], a_min=a_min, a_max=a_max,
                                b_min=b_min, b_max=b_max, clip=True),
                             transforms.CropForegroundd(keys="image", source_key="image", select_fn=threshold),
                             transforms.Resized(keys="image", mode="bilinear", align_corners=True,
                                    spatial_size=(384, 384, 96)),
                            VoCoAugmentation( aug=True)
                            ])



In [9]:
import multiprocessing as mp

mp.set_start_method('spawn', force=True)

In [10]:
train_ds = data.PersistentDataset(data=datalist,
                                     transform=train_transforms,
                                     pickle_protocol=pickle.HIGHEST_PROTOCOL,
                                     cache_dir='/data/saved/voco_cache')
train_sampler = None
train_loader = data.DataLoader(
        train_ds, batch_size=batch_size, num_workers=num_workers, sampler=train_sampler, shuffle=True,
        drop_last=True, pin_memory=True
    )

for step, batch in enumerate(train_loader):
#             t1 = time()
#             print(batch)
            img, labels, crops = batch

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data/anaconda3/envs/voco/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data/anaconda3/envs/voco/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'VoCoAugmentation' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 3342393) exited unexpectedly

In [12]:
for step, batch in enumerate(train_loader):
    print(step)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data/anaconda3/envs/voco/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data/anaconda3/envs/voco/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'VoCoAugmentation' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 3373255) exited unexpectedly